# Data Extraction Notebook
本ノートブックでは不動産のマスターデータから東京都の物件に絞り、クリーニングされた12万行のデータを作成する。

In [169]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## マスターデータの読み込み
一度読み込んだ後サンプリングをする。実行時間短縮の為コメントアウトしてある。

In [170]:
RENT_CONV_PATH = r"C:\Users\Issei\Desktop\Class Documents\KEIO 2024 Fall\Data Science\Datasets\rent_converted.tsv"
SAMPLED_RENT_PATH = r"C:\Users\Issei\Desktop\Class Documents\KEIO 2024 Fall\Data Science\Datasets\Output\rent_sampled.csv"
COLUMN_NAMES = [
    "物件ID", "作成日時", "公開日時", "修正日時", "自社物フラグ", "物件種別", "総戸数/総区画数", "空き物件数", "郵便番号",
    "都道府県", "市区郡町村", "路線1", "駅1", "バス停名1", "バス時間1", "徒歩距離1", "路線2", "駅2", "バス停名2", "バス時間2",
    "徒歩距離2", "その他交通", "用途地域", "都市計画", "建物構造", "建物面積/専有面積", "建物階数(地上)", "建物階数(地下)", "築年月",
    "新築・未入居フラグ", "管理人", "部屋階数", "向き", "間取部屋数", "間取部屋種類", "間取り備考", "物件の特徴", "賃料/価格",
    "共益費/管理費", "賃料＋管理費", "礼金", "敷金", "保証金", "更新料", "契約期間(年)", "契約期間(月)", "契約期間(区分)",
    "その他費用名目1", "その他費用1", "その他費用名目2", "その他費用2", "その他費用名目3", "その他費用3",
    "駐車場料金", "駐車場区分", "駐車場距離", "駐車場空き台数", "駐車場備考", "現況", "引渡/入居時期", "引渡/入居年月",
    "引渡/入居旬", "小学校名", "小学校距離", "中学校名", "中学校距離", "コンビニ距離", "スーパー距離", "総合病院距離",
    "取引態様", "仲介手数料"
]
# master_df = pd.read_csv(RENT_CONV_PATH, sep='\t', names=COLUMN_NAMES, header=None)
master_df.head()

,物件ID,作成日時,公開日時,修正日時,自社物フラグ,物件種別,総戸数/総区画数,空き物件数,郵便番号,都道府県,...,引渡/入居旬,小学校名,小学校距離,中学校名,中学校距離,コンビニ距離,スーパー距離,総合病院距離,取引態様,仲介手数料
0,000001a8a09bac529563c00e71099276,2015-05-16,2015-05-16,2015-09-06,1,3102,NaN,NaN,230-0012,14,...,NaN,NaN,NaN,NaN,NaN,270.0,NaN,NaN,5,31860.0
1,000003d67c168129876425c22a106dae,2015-08-07,2015-08-07,2015-09-06,0,3102,21.0,NaN,260-0843,12,...,1.0,NaN,NaN,NaN,NaN,300.0,110.0,NaN,6,NaN
2,00000441bc94d33c6889477ea2b09941,2015-08-01,2015-08-01,2015-09-05,0,3101,NaN,NaN,114-0023,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
3,0000055b093c4209d164a7f2204eff32,2015-07-21,2015-07-21,2015-09-06,1,3103,NaN,NaN,432-8052,22,...,NaN,市立可美小学校,1600.0,市立可美中学校,2100.0,390.0,NaN,NaN,5,42120.0
4,0000081549d99cf1e3f5be593e560799,2015-02-12,2015-05-16,2015-09-06,1,3101,NaN,0.0,065-0009,1,...,NaN,札幌市立苗穂小学校,605.0,札幌市立美香保中学校,1057.0,121.0,267.0,NaN,5,NaN


In [171]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5334370 entries, 0 to 5334369
Data columns (total 71 columns):
 #   Column     Dtype  
---  ------     -----  
 0   物件ID       object 
 1   作成日時       object 
 2   公開日時       object 
 3   修正日時       object 
 4   自社物フラグ     int64  
 5   物件種別       int64  
 6   総戸数/総区画数   float64
 7   空き物件数      float64
 8   郵便番号       object 
 9   都道府県       int64  
 10  市区郡町村      int64  
 11  路線1        float64
 12  駅1         float64
 13  バス停名1      object 
 14  バス時間1      float64
 15  徒歩距離1      float64
 16  路線2        float64
 17  駅2         float64
 18  バス停名2      object 
 19  バス時間2      float64
 20  徒歩距離2      float64
 21  その他交通      object 
 22  用途地域       float64
 23  都市計画       float64
 24  建物構造       float64
 25  建物面積/専有面積  float64
 26  建物階数(地上)   float64
 27  建物階数(地下)   float64
 28  築年月        float64
 29  新築・未入居フラグ  int64  
 30  管理人        float64
 31  部屋階数       float64
 32  向き         float64
 33  間取部屋数      float64
 34  間取部屋種類     float64

## 東京物件のフィルタリングとサンプリング

In [172]:
# Filter out to get only Tokyo data
try:
    sampled_df = master_df[master_df['都道府県']==13].sample(130000, random_state=0) # 13 is the code for Tokyo
except:
    sampled_df = pd.read_csv(SAMPLED_RENT_PATH)
smapled_df = sampled_df.reset_index(drop=True)
sampled_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130000 entries, 1863800 to 1142576
Data columns (total 71 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   物件ID       130000 non-null  object 
 1   作成日時       130000 non-null  object 
 2   公開日時       130000 non-null  object 
 3   修正日時       130000 non-null  object 
 4   自社物フラグ     130000 non-null  int64  
 5   物件種別       130000 non-null  int64  
 6   総戸数/総区画数   35251 non-null   float64
 7   空き物件数      17936 non-null   float64
 8   郵便番号       127961 non-null  object 
 9   都道府県       130000 non-null  int64  
 10  市区郡町村      130000 non-null  int64  
 11  路線1        130000 non-null  float64
 12  駅1         130000 non-null  float64
 13  バス停名1      5357 non-null    object 
 14  バス時間1      24784 non-null   float64
 15  徒歩距離1      130000 non-null  float64
 16  路線2        115937 non-null  float64
 17  駅2         115937 non-null  float64
 18  バス停名2      5262 non-null    object 
 19  バス時間2      21962 non-

In [173]:
# Save Sampled_df to a new csv file
# sampled_df.to_csv(r"C:\Users\Issei\Desktop\Class Documents\KEIO 2024 Fall\Data Science\Datasets\Output\rent_sampled.csv", sep=',', index=False)

## データクリーニング EDA.jpynbを参照
### 特徴量エンジニアリング

In [174]:
sampled_df['バス停有り'] = sampled_df['バス停名1'].notnull() | sampled_df['バス停名2'].notnull()

### 欠損値の処理

In [175]:
# 賃料+管理費は賃料/価格と共益費/管理費の合計なので削除（データ漏れがあるため）
cols_drop = ['間取り備考', '都市計画', '駐車場備考', '用途地域', 'バス停名1', 'バス停名2', 'バス時間1','バス時間2',
              'その他費用名目1', 'その他費用名目2', 'その他費用名目3', '駐車場空き台数', '管理人', '引渡/入居年月',
              '引渡/入居旬', '物件の特徴', '建物階数(地下)', '仲介手数料', '保証金', '更新料', '中学校名', '小学校名',
              '中学校距離', '小学校距離', '総合病院距離', '総戸数/総区画数', '賃料＋管理費', 'その他交通', '契約期間(区分)',
              '作成日時', '公開日時', '修正日時', '郵便番号']

sampled_df = sampled_df.drop(columns=cols_drop, axis=1)

In [176]:
# Impute missing values (simple imputer)
sampled_df[['その他費用1', 'その他費用2', 'その他費用3']] = sampled_df[['その他費用1', 'その他費用2', 'その他費用3']].fillna(0)
sampled_df['駐車場料金'] = sampled_df['駐車場料金'].fillna(0)
sampled_df[['駅1', '駅2', '路線1', '路線2', '徒歩距離2']] = sampled_df[['駅1', '駅2', '路線1', '路線2', '徒歩距離2']].fillna(-1)
sampled_df[['駐車場区分', '現況']] = sampled_df[['駐車場区分', '現況']].fillna(-1) # -1 is unknown
sampled_df['空き物件数'] = sampled_df['空き物件数'].fillna(1)

cols_mode = [['徒歩距離1', '築年月', '建物階数(地上)', '部屋階数', '向き', '間取部屋数', '間取部屋種類', '建物構造']]
for col in cols_mode:
    sampled_df[col] = sampled_df[col].fillna(sampled_df[col].mode().iloc[0])
sampled_df['建物面積/専有面積'] = sampled_df['建物面積/専有面積'].fillna(sampled_df['建物面積/専有面積'].mean())

In [177]:
# その他費用を一つに纏めて新しい列を作成
sampled_df['その他費用'] = sampled_df['その他費用1'] + sampled_df['その他費用2'] + sampled_df['その他費用3']
sampled_df = sampled_df.drop(columns=['その他費用1', 'その他費用2', 'その他費用3'], axis=1)

In [178]:
# Impute missing values (Advanced)
def label_distance(distance):
    if distance < 0:
        return -1 # unknown
    elif distance < 100:
        return 1 # close
    elif distance < 300:
        return 2 # medium
    elif distance < 700:
        return 3 # far
    else:
        return 4 # very far
    
sampled_df[['コンビニ距離', 'スーパー距離', '駐車場距離']] = sampled_df[['コンビニ距離', 'スーパー距離', '駐車場距離']].fillna(-1)
sampled_df['コンビニ距離'] = sampled_df['コンビニ距離'].apply(label_distance)
sampled_df['スーパー距離'] = sampled_df['スーパー距離'].apply(label_distance)
sampled_df['駐車場距離'] = sampled_df['駐車場距離'].apply(label_distance)

sampled_df.loc[sampled_df['契約期間(月)'].isnull() & (sampled_df['契約期間(年)'] > 0), '契約期間(月)'] = (sampled_df['契約期間(年)'] * 12)
sampled_df.loc[sampled_df['契約期間(年)'].isnull() & (sampled_df['契約期間(月)'] > 0), '契約期間(年)'] = (sampled_df['契約期間(月)'] / 12)
median_contract = sampled_df['契約期間(月)'].median()
sampled_df['契約期間(月)'] = sampled_df['契約期間(月)'].fillna(median_contract)
sampled_df['契約期間(年)'] = sampled_df['契約期間(年)'].fillna(median_contract * 12)
#同じ情報を含んでいるので契約期間(年)を削除
sampled_df.drop('契約期間(月)', axis=1, inplace=True)

In [179]:
# Check null values
sampled_df.isnull().sum().loc[sampled_df.isnull().sum() > 0]

Series([], dtype: int64)

In [180]:
sampled_df.reset_index(drop=True, inplace=True)

In [181]:
sampled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 36 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   物件ID       130000 non-null  object 
 1   自社物フラグ     130000 non-null  int64  
 2   物件種別       130000 non-null  int64  
 3   空き物件数      130000 non-null  float64
 4   都道府県       130000 non-null  int64  
 5   市区郡町村      130000 non-null  int64  
 6   路線1        130000 non-null  float64
 7   駅1         130000 non-null  float64
 8   徒歩距離1      130000 non-null  float64
 9   路線2        130000 non-null  float64
 10  駅2         130000 non-null  float64
 11  徒歩距離2      130000 non-null  float64
 12  建物構造       130000 non-null  float64
 13  建物面積/専有面積  130000 non-null  float64
 14  建物階数(地上)   130000 non-null  float64
 15  築年月        130000 non-null  float64
 16  新築・未入居フラグ  130000 non-null  int64  
 17  部屋階数       130000 non-null  float64
 18  向き         130000 non-null  float64
 19  間取部屋数      130000 non-n

In [182]:
sampled_df.head()

,物件ID,自社物フラグ,物件種別,空き物件数,都道府県,市区郡町村,路線1,駅1,徒歩距離1,路線2,...,駐車場料金,駐車場区分,駐車場距離,現況,引渡/入居時期,コンビニ距離,スーパー距離,取引態様,バス停有り,その他費用
0,597c3e31ff67908dc29e8ead4ee76f75,1,3101,1.0,13,214,114.0,767.0,720.0,114.0,...,12000.0,1.0,-1,-1.0,2,-1,-1,5,False,0.0
1,2fa77b2075c23bbc6968576c4360e014,0,3101,1.0,13,111,894.0,5141.0,480.0,93.0,...,35000.0,3.0,2,2.0,1,1,2,6,False,64800.0
2,c7a0a50a85eda8a6453339c55c9da575,0,3102,1.0,13,123,194.0,1929.0,240.0,194.0,...,0.0,4.0,-1,-1.0,1,-1,-1,6,True,19440.0
3,52d361d62cfc67eb00bc0496b8eb46a6,0,3101,1.0,13,122,129.0,1039.0,160.0,860.0,...,0.0,-1.0,-1,2.0,1,1,2,6,False,16200.0
4,9e504ad466b768d9b000199af98e2b1d,1,3102,1.0,13,212,863.0,4952.0,400.0,110.0,...,8640.0,1.0,-1,2.0,1,-1,-1,6,False,0.0


## トレインとテストデータの作成と保存

In [183]:
train_tokyo_df = sampled_df.sample(frac=0.8, random_state=0)
answer_tokyo_df = sampled_df.drop(train_tokyo_df.index)['賃料/価格']
test_tokyo_df = sampled_df.drop(train_tokyo_df.index).drop('賃料/価格', axis=1)

In [184]:
train_tokyo_df.to_csv(r"C:\Users\Issei\Desktop\Class Documents\KEIO 2024 Fall\Data Science\Datasets\Output\train_tokyo.csv", sep=',', index=False)
test_tokyo_df.to_csv(r"C:\Users\Issei\Desktop\Class Documents\KEIO 2024 Fall\Data Science\Datasets\Output\test_tokyo.csv", sep=',', index=False)
answer_tokyo_df.to_csv(r"C:\Users\Issei\Desktop\Class Documents\KEIO 2024 Fall\Data Science\Datasets\Output\answer_tokyo.csv", sep=',', index=False)

In [3]:
import chardet

# Path to your CSV file
file_path = r"C:\Users\Issei\Desktop\Class Documents\KEIO 2024 Fall\Data Science\Datasets\Output\train_tokyo.csv"

# Read the file in binary mode
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

# Display the detected encoding
print(result)

ModuleNotFoundError: No module named 'chardet'

: 